In [49]:
from pyspark.sql import SparkSession, functions as f

In [57]:
spark = SparkSession.builder.getOrCreate()

In [4]:
df = spark.read.csv("house_price_prediction_dataset.csv", header=True, inferSchema=True)

In [5]:
df.createOrReplaceTempView("houses")

##### SCOPE
Here I try to get the requests with spark methods and sql queries. The requests according to `house_price_simple_analysis.py` file:
01. Schema of the dataset.
02. Row count of the dataset.
03. Column count of the dataset.
04. Describes of descriptive stats.
05. Min of descriptive stats.
06. Max of descriptive stats.
07. Null control for some columns.
08. Fill null rows.
09. Bedrooms biggest than three.
10. Year built < 2010.
11. Location count.
12. Price group by location.
13. Price per square.
14. Order by price.
15. Price by area.
16. Adding has_garage column to dataset.<br><br><br>
***  
##### TODO

- [x] Schema of the dataset.
- [x] Row count of the dataset.
- [x] Column count of the dataset.
- [x] Describes of descriptive stats.
- [x] Min of descriptive stats.
- [x] Max of descriptive stats.
- [x] Null control for some columns.
- [x] Fill null rows.
- [x] Bedrooms biggest than three.
- [x] Year built < 2010.
- [x] Location count.
- [x] Price group by location.
- [x] Price per square.
- [x] Order by price.
- [x] Price by area.
- [x] Adding has_garage column to dataset.

In [12]:
schema_of_dataset_py = df.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- Area: integer (nullable = true)
 |-- Bedrooms: integer (nullable = true)
 |-- Bathrooms: integer (nullable = true)
 |-- Floors: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- Location: string (nullable = true)
 |-- Condition: string (nullable = true)
 |-- Garage: string (nullable = true)
 |-- Price: integer (nullable = true)



In [13]:
schema_of_dataset_sql = spark.sql("DESCRIBE houses;").show()

+---------+---------+-------+
| col_name|data_type|comment|
+---------+---------+-------+
|       Id|      int|   NULL|
|     Area|      int|   NULL|
| Bedrooms|      int|   NULL|
|Bathrooms|      int|   NULL|
|   Floors|      int|   NULL|
|YearBuilt|      int|   NULL|
| Location|   string|   NULL|
|Condition|   string|   NULL|
|   Garage|   string|   NULL|
|    Price|      int|   NULL|
+---------+---------+-------+



In [33]:
count_of_dataset_py = df.count()
print("Row Count of the Dataset: ",count_of_dataset_py)

Row Count of the Dataset:  2000


In [19]:
count_of_dataset_sql = spark.sql(
    """
    SELECT COUNT(*) AS row_count FROM houses;
    """
    ).show()

+---------+
|row_count|
+---------+
|     2000|
+---------+



In [34]:
column_count_of_dataset_py = len(df.columns)
print("Column Count of the Dataset: ", column_count_of_dataset_py)

Column Count of the Dataset:  10


In [29]:
column_count_of_dataset_sql = spark.sql(
    """
    SELECT SIZE(ARRAY(*)) AS column_count FROM houses LIMIT 1;
    """
).show()

+------------+
|column_count|
+------------+
|          10|
+------------+



In [14]:
df.describe("Bedrooms", "Bathrooms", "Floors", "YearBuilt", "Price").show()

24/12/17 20:30:57 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----------------+------------------+------------------+-----------------+-----------------+
|summary|         Bedrooms|         Bathrooms|            Floors|        YearBuilt|            Price|
+-------+-----------------+------------------+------------------+-----------------+-----------------+
|  count|             2000|              2000|              2000|             2000|             2000|
|   mean|           3.0035|            2.5525|            1.9935|         1961.446|       537676.855|
| stddev|1.424606086344792|1.1089899365366986|0.8091879525618783|35.92669547458914|276428.8457191392|
|    min|                1|                 1|                 1|             1900|            50005|
|    max|                5|                 4|                 3|             2023|           999656|
+-------+-----------------+------------------+------------------+-----------------+-----------------+



In [22]:
spark.sql(
    """
    SELECT
        'Bedrooms' AS summary,
        COUNT(Bedrooms) AS count,
        AVG(Bedrooms) AS mean,
        STDDEV(Bedrooms) AS stddev,
        MIN(Bedrooms) AS min,
        MAX(Bedrooms) AS max
    FROM houses
    UNION
    SELECT
        'Bathrooms' AS summary,
        COUNT(Bathrooms) AS count,
        AVG(Bathrooms) AS mean,
        STDDEV(Bathrooms) AS stddev,
        MIN(Bathrooms) AS min,
        MAX(Bathrooms) AS max
    FROM houses
    UNION
    SELECT
        'Floors' AS summary,
        COUNT(Floors) AS count,
        AVG(Floors) AS mean,
        STDDEV(Floors) AS stddev,
        MIN(Floors) AS min,
        MAX(Floors) AS max
    FROM houses
    UNION
    SELECT
        'YearBuilt' AS summary,
        COUNT(YearBuilt) AS count,
        AVG(YearBuilt) AS mean,
        STDDEV(YearBuilt) AS stddev,
        MIN(YearBuilt) AS min,
        MAX(YearBuilt) AS max
    FROM houses
    UNION
    SELECT
        'Price' AS summary,
        COUNT(Price) AS count,
        AVG(Price) AS mean,
        STDDEV(Price) AS stddev,
        MIN(Price) AS min,
        MAX(Price) AS max
    FROM houses;
    """
).show()

+---------+-----+----------+------------------+-----+------+
|  summary|count|      mean|            stddev|  min|   max|
+---------+-----+----------+------------------+-----+------+
| Bedrooms| 2000|    3.0035| 1.424606086344792|    1|     5|
|Bathrooms| 2000|    2.5525|1.1089899365366986|    1|     4|
|   Floors| 2000|    1.9935|0.8091879525618783|    1|     3|
|YearBuilt| 2000|  1961.446| 35.92669547458914| 1900|  2023|
|    Price| 2000|537676.855| 276428.8457191392|50005|999656|
+---------+-----+----------+------------------+-----+------+



In [32]:
df.select("Bedrooms", "Bathrooms", "Floors", "YearBuilt", "Price").groupBy() \
  .min("Bedrooms", "Bathrooms", "Floors", "YearBuilt", "Price").show()

+-------------+--------------+-----------+--------------+----------+
|min(Bedrooms)|min(Bathrooms)|min(Floors)|min(YearBuilt)|min(Price)|
+-------------+--------------+-----------+--------------+----------+
|            1|             1|          1|          1900|     50005|
+-------------+--------------+-----------+--------------+----------+



In [33]:
spark.sql(
    """
    SELECT MIN(Bedrooms), MIN(Bathrooms), MIN(Floors), MIN(YearBuilt), MIN(Price) FROM houses;
    """
).show()

+-------------+--------------+-----------+--------------+----------+
|min(Bedrooms)|min(Bathrooms)|min(Floors)|min(YearBuilt)|min(Price)|
+-------------+--------------+-----------+--------------+----------+
|            1|             1|          1|          1900|     50005|
+-------------+--------------+-----------+--------------+----------+



In [34]:
df.select("Bedrooms", "Bathrooms", "Floors", "YearBuilt", "Price").groupBy() \
  .max("Bedrooms", "Bathrooms", "Floors", "YearBuilt", "Price").show()

+-------------+--------------+-----------+--------------+----------+
|max(Bedrooms)|max(Bathrooms)|max(Floors)|max(YearBuilt)|max(Price)|
+-------------+--------------+-----------+--------------+----------+
|            5|             4|          3|          2023|    999656|
+-------------+--------------+-----------+--------------+----------+



In [35]:
spark.sql(
    """
    SELECT MAX(Bedrooms), MAX(Bathrooms), MAX(Floors), MAX(YearBuilt), MAX(Price) FROM houses;
    """
).show()

+-------------+--------------+-----------+--------------+----------+
|max(Bedrooms)|max(Bathrooms)|max(Floors)|max(YearBuilt)|max(Price)|
+-------------+--------------+-----------+--------------+----------+
|            5|             4|          3|          2023|    999656|
+-------------+--------------+-----------+--------------+----------+



In [14]:
columns = ["Id", "Area", "Bedrooms", "Bathrooms", "Floors", "YearBuilt", "Location", "Condition", "Garage", "Price"]
for col in columns:
    result = df.filter(df[col].isNull()).count()
    print(f"{col}: {result}")

Id: 0
Area: 0
Bedrooms: 0
Bathrooms: 0
Floors: 0
YearBuilt: 0
Location: 0
Condition: 0
Garage: 0
Price: 0


In [ ]:
spark.sql(
    """
    SELECT 
        COUNT(Id) IS NOT NULL,
        Area IS NOT NULL,
        Bedrooms IS NOT NULL, 
        Bathrooms IS NOT NULL, 
        Floors IS NOT NULL, 
        YearBuilt IS NOT NULL, 
        Location IS NOT NULL, 
        Condition IS NOT NULL, 
        Garage IS NOT NULL, 
        Price IS NOT NULL
    FROM houses;
    """
).show()

In [26]:
id = df.select("Id").groupBy().max("Id").collect()[0][0]
area = df.select("Area").groupBy().avg("Area").collect()[0][0]
bedrooms = df.select("Bedrooms").groupBy().avg("Bedrooms").collect()[0][0]
bathrooms = df.select("Bathrooms").groupBy().avg("Bathrooms").collect()[0][0]
floors = df.select("Floors").groupBy().avg("Floors").collect()[0][0]
yearbuilt = df.select("YearBuilt").groupBy().avg("YearBuilt").collect()[0][0]
price = df.select("Price").groupBy().avg("Price").collect()[0][0]

In [27]:
df.na.fill({
    'Id': id+1,
    'Area': area,
    'Bedrooms': bedrooms,
    "Bathrooms": bathrooms,
    "Floors": floors,
    "YearBuilt": yearbuilt,
    'Location': 'unknown',
    'Condition': 'unknown',
    'Garage': 'unknown',
    "Price": price
}).show(10)

+---+----+--------+---------+------+---------+--------+---------+------+------+
| Id|Area|Bedrooms|Bathrooms|Floors|YearBuilt|Location|Condition|Garage| Price|
+---+----+--------+---------+------+---------+--------+---------+------+------+
|  1|1360|       5|        4|     3|     1970|Downtown|Excellent|    No|149919|
|  2|4272|       5|        4|     3|     1958|Downtown|Excellent|    No|424998|
|  3|3592|       2|        2|     3|     1938|Downtown|     Good|    No|266746|
|  4| 966|       4|        2|     2|     1902|Suburban|     Fair|   Yes|244020|
|  5|4926|       1|        4|     2|     1975|Downtown|     Fair|   Yes|636056|
|  6|3944|       1|        2|     1|     1906|   Urban|     Poor|    No| 93262|
|  7|3671|       1|        1|     2|     1948|   Rural|     Poor|   Yes|448722|
|  8|3419|       2|        4|     1|     1925|Suburban|     Good|   Yes|594893|
|  9| 630|       2|        2|     1|     1932|   Rural|     Poor|   Yes|652878|
| 10|2185|       3|        3|     1|    

##### A point:
There is no chance here to coding a sql query can insert data to null rows. I tried a code block but I taked Unsupported Operation Error directly. Because of that, we can use Delta Lake here. But it will take a lot of time, so maybe I will try later with Delta Lake.  

For now I just show the empty lines.

In [13]:
spark.sql(
    """
    SELECT Id, Area, Bedrooms, Bathrooms, Floors, YearBuilt, Price FROM houses WHERE Id IS NULL;
    """
).show()

+---+----+--------+---------+------+---------+-----+
| Id|Area|Bedrooms|Bathrooms|Floors|YearBuilt|Price|
+---+----+--------+---------+------+---------+-----+
+---+----+--------+---------+------+---------+-----+



In [30]:
bedrooms_biggest_than_three = df.filter(df.Bedrooms > 3).count()
print(bedrooms_biggest_than_three)

808


In [15]:
spark.sql(
    """
    SELECT COUNT(Bedrooms) AS bedrooms_biggest_than_three FROM houses WHERE Bedrooms > 3;
    """
).show()

+---------------------------+
|bedrooms_biggest_than_three|
+---------------------------+
|                        808|
+---------------------------+



In [31]:
year_built_less_than_2010 = df.filter(df.YearBuilt < 2010).count()
print(year_built_less_than_2010)

1779


In [17]:
spark.sql(
    """
    SELECT COUNT(YearBuilt) AS year_built_less_than_2010 FROM houses WHERE YearBuilt < 2010;
    """
).show()

+-------------------------+
|year_built_less_than_2010|
+-------------------------+
|                     1779|
+-------------------------+



In [48]:
df.groupBy("Location").count().show()

+--------+-----+
|Location|count|
+--------+-----+
|   Urban|  485|
|Suburban|  483|
|Downtown|  558|
|   Rural|  474|
+--------+-----+



In [19]:
spark.sql(
    """
    SELECT Location, COUNT(Location) AS Count FROM houses GROUP BY Location;
    """
).show()

+--------+-----+
|Location|Count|
+--------+-----+
|   Urban|  485|
|Suburban|  483|
|Downtown|  558|
|   Rural|  474|
+--------+-----+



In [53]:
df.groupBy("Location").avg("Price").show()

+--------+------------------+
|Location|        avg(Price)|
+--------+------------------+
|   Urban|518963.54845360824|
|Suburban| 557416.3333333334|
|Downtown| 536059.6612903225|
|   Rural| 538613.9430379746|
+--------+------------------+



In [21]:
spark.sql(
    """
    SELECT Location, AVG(Price) FROM houses GROUP BY Location;
    """
).show()

+--------+------------------+
|Location|        avg(Price)|
+--------+------------------+
|   Urban|518963.54845360824|
|Suburban| 557416.3333333334|
|Downtown| 536059.6612903225|
|   Rural| 538613.9430379746|
+--------+------------------+



In [31]:
df.withColumn("price_per_square", (df.Price/df.Area)).show(5)

+---+----+--------+---------+------+---------+--------+---------+------+------+------------------+
| Id|Area|Bedrooms|Bathrooms|Floors|YearBuilt|Location|Condition|Garage| Price|  price_per_square|
+---+----+--------+---------+------+---------+--------+---------+------+------+------------------+
|  1|1360|       5|        4|     3|     1970|Downtown|Excellent|    No|149919|110.23455882352941|
|  2|4272|       5|        4|     3|     1958|Downtown|Excellent|    No|424998| 99.48455056179775|
|  3|3592|       2|        2|     3|     1938|Downtown|     Good|    No|266746| 74.26113585746103|
|  4| 966|       4|        2|     2|     1902|Suburban|     Fair|   Yes|244020| 252.6086956521739|
|  5|4926|       1|        4|     2|     1975|Downtown|     Fair|   Yes|636056|129.12220868859114|
+---+----+--------+---------+------+---------+--------+---------+------+------+------------------+
only showing top 5 rows



In [38]:
spark.sql(
    """
    SELECT *, (Price/Area) AS price_per_square FROM houses;
    """
).show(5)

+---+----+--------+---------+------+---------+--------+---------+------+------+------------------+
| Id|Area|Bedrooms|Bathrooms|Floors|YearBuilt|Location|Condition|Garage| Price|  price_per_square|
+---+----+--------+---------+------+---------+--------+---------+------+------+------------------+
|  1|1360|       5|        4|     3|     1970|Downtown|Excellent|    No|149919|110.23455882352941|
|  2|4272|       5|        4|     3|     1958|Downtown|Excellent|    No|424998| 99.48455056179775|
|  3|3592|       2|        2|     3|     1938|Downtown|     Good|    No|266746| 74.26113585746103|
|  4| 966|       4|        2|     2|     1902|Suburban|     Fair|   Yes|244020| 252.6086956521739|
|  5|4926|       1|        4|     2|     1975|Downtown|     Fair|   Yes|636056|129.12220868859114|
+---+----+--------+---------+------+---------+--------+---------+------+------+------------------+
only showing top 5 rows



In [45]:
df.orderBy(df.Price.asc()).show(5)

+----+----+--------+---------+------+---------+--------+---------+------+-----+
|  Id|Area|Bedrooms|Bathrooms|Floors|YearBuilt|Location|Condition|Garage|Price|
+----+----+--------+---------+------+---------+--------+---------+------+-----+
|1168|4260|       2|        2|     1|     1907|   Rural|     Fair|   Yes|50005|
| 837|2871|       4|        2|     1|     1914|Suburban|     Poor|    No|50064|
|1729| 509|       4|        1|     2|     1993|   Urban|     Fair|    No|51082|
|1593|4368|       5|        4|     2|     1976|Downtown|     Fair|    No|51845|
|1464|4138|       3|        4|     1|     1960|   Urban|     Fair|    No|52024|
+----+----+--------+---------+------+---------+--------+---------+------+-----+
only showing top 5 rows



In [40]:
spark.sql(
    """
    SELECT * FROM houses ORDER BY Price;
    """
).show(5)

+----+----+--------+---------+------+---------+--------+---------+------+-----+
|  Id|Area|Bedrooms|Bathrooms|Floors|YearBuilt|Location|Condition|Garage|Price|
+----+----+--------+---------+------+---------+--------+---------+------+-----+
|1168|4260|       2|        2|     1|     1907|   Rural|     Fair|   Yes|50005|
| 837|2871|       4|        2|     1|     1914|Suburban|     Poor|    No|50064|
|1729| 509|       4|        1|     2|     1993|   Urban|     Fair|    No|51082|
|1593|4368|       5|        4|     2|     1976|Downtown|     Fair|    No|51845|
|1464|4138|       3|        4|     1|     1960|   Urban|     Fair|    No|52024|
+----+----+--------+---------+------+---------+--------+---------+------+-----+
only showing top 5 rows



In [44]:
df.orderBy(["Area", "Price"], ascending=[False, True]).show(5)

+----+----+--------+---------+------+---------+--------+---------+------+------+
|  Id|Area|Bedrooms|Bathrooms|Floors|YearBuilt|Location|Condition|Garage| Price|
+----+----+--------+---------+------+---------+--------+---------+------+------+
| 206|4999|       4|        2|     1|     1967|Downtown|Excellent|    No|139460|
|1123|4997|       2|        2|     2|     1918|   Urban|Excellent|    No|286887|
| 161|4996|       1|        1|     1|     2005|Downtown|     Fair|    No|369733|
| 233|4996|       2|        1|     3|     1943|   Urban|     Fair|   Yes|784184|
| 300|4995|       2|        1|     3|     1925|   Urban|     Good|    No|286028|
+----+----+--------+---------+------+---------+--------+---------+------+------+
only showing top 5 rows



In [48]:
spark.sql(
    """
    SELECT * FROM houses ORDER BY Area DESC, Price;
    """
).show(5)

+----+----+--------+---------+------+---------+--------+---------+------+------+
|  Id|Area|Bedrooms|Bathrooms|Floors|YearBuilt|Location|Condition|Garage| Price|
+----+----+--------+---------+------+---------+--------+---------+------+------+
| 206|4999|       4|        2|     1|     1967|Downtown|Excellent|    No|139460|
|1123|4997|       2|        2|     2|     1918|   Urban|Excellent|    No|286887|
| 161|4996|       1|        1|     1|     2005|Downtown|     Fair|    No|369733|
| 233|4996|       2|        1|     3|     1943|   Urban|     Fair|   Yes|784184|
| 300|4995|       2|        1|     3|     1925|   Urban|     Good|    No|286028|
+----+----+--------+---------+------+---------+--------+---------+------+------+
only showing top 5 rows



In [53]:
df.withColumn("has_garage", f.when(df.Garage == "Yes", True).otherwise(False)).show(5)

+---+----+--------+---------+------+---------+--------+---------+------+------+----------+
| Id|Area|Bedrooms|Bathrooms|Floors|YearBuilt|Location|Condition|Garage| Price|has_garage|
+---+----+--------+---------+------+---------+--------+---------+------+------+----------+
|  1|1360|       5|        4|     3|     1970|Downtown|Excellent|    No|149919|     false|
|  2|4272|       5|        4|     3|     1958|Downtown|Excellent|    No|424998|     false|
|  3|3592|       2|        2|     3|     1938|Downtown|     Good|    No|266746|     false|
|  4| 966|       4|        2|     2|     1902|Suburban|     Fair|   Yes|244020|      true|
|  5|4926|       1|        4|     2|     1975|Downtown|     Fair|   Yes|636056|      true|
+---+----+--------+---------+------+---------+--------+---------+------+------+----------+
only showing top 5 rows



In [56]:
spark.sql(
    """
    SELECT CASE 
        WHEN Garage = 'Yes' THEN True
        ELSE False
        END AS has_garage, Garage FROM houses;
    """
).show(5)

+----------+------+
|has_garage|Garage|
+----------+------+
|     false|    No|
|     false|    No|
|     false|    No|
|      true|   Yes|
|      true|   Yes|
+----------+------+
only showing top 5 rows

